# Market basket analysis(MBA) for predicting products bought together

In [1]:
import kagglehub
import shutil
import os
import pandas as pd
import random

/Users/azeezatmustapha/Desktop/shopper/backend/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version
#path = kagglehub.dataset_download("psparks/instacart-market-basket-analysis")

# Move it to local project folder
#destination = "./data/instacart"
#os.makedirs(destination, exist_ok=True)
#shutil.move(path, destination)

100%|████████████████████████████████████████| 197M/197M [00:07<00:00, 29.1MB/s]

Extracting files...


'./data/instacart/1'

In [3]:
#Get the 1000 most ordered product ids from the orders
ordered_products = pd.read_csv('data/instacart/order_products__prior.csv')
top_ids = ordered_products['product_id'].value_counts().head(1000).index
top_ids

Index([24852, 13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209, 27845,
       ...
        4945, 13966,  7746, 35199, 45190, 18023, 20378, 43014,  3339, 27020],
      dtype='int64', name='product_id', length=1000)

In [54]:
len(unique_orders)

2963012

In [4]:
#Filter orders to only contain the 1,000 most ordered products
order_filter = ordered_products[ordered_products['product_id'].isin(top_ids)]
print(f"length of order_filter: {len(order_filter)}\n")

# Get list of unique order IDs and sample them
unique_orders = order_filter['order_id'].unique()
print(f"length of unique_orders: {len(unique_orders)}\n")

# Sampling 500k of the millions of unique orders
sampled_orders = pd.Series(unique_orders).sample(n=500000, random_state=42)

# Filter the dataframe to only include these 500k orders
subset_df = order_filter[order_filter['order_id'].isin(sampled_orders)]

#Filter orders to only contain orders with more than 1 product
order_counts = subset_df.groupby('order_id').size()
valid_orders = order_counts[order_counts > 1].index
print(f"length of valid_orders: {len(valid_orders)}\n")

## Filtering is linked to the main df, adding .copy() makes it a df of itself, not linked to the bigger dataframe
order_df = subset_df[subset_df['order_id'].isin(valid_orders)].copy()
print(f"length of final order_df: {len(order_df)}.\n Reduced by sampling 500k unique others from {len(unique_orders)} unique orders.\n Working with {len(order_df)} samples instead of {len(order_filter)}")

length of order_filter: 17515025

length of unique_orders: 2963012

length of valid_orders: 432188

length of final order_df: 2884578.
 Reduced by sampling 500k unique others from 2963012 unique orders.
 Working with 2884578 samples instead of 17515025


In [5]:
#Verifying the Sample is a true representation

print(order_filter['product_id'].value_counts(normalize=True).head(5))
print(order_df['product_id'].value_counts(normalize=True).head(5))

product_id
24852    0.026981
13176    0.021664
21137    0.015112
21903    0.013812
47209    0.012194
Name: proportion, dtype: float64
product_id
24852    0.027045
13176    0.021542
21137    0.015117
21903    0.013978
47209    0.012266
Name: proportion, dtype: float64


In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules

In [7]:
## 1 hot encoding
# Convert to a list of lists (one list of products per order)
transactions = order_df.groupby('order_id')['product_id'].apply(list).values.tolist()

In [8]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
basket_sets = pd.DataFrame(te_ary, columns=te.columns_)

In [9]:
frequent_itemsets = fpgrowth(basket_sets, min_support=0.005, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.093295,(21903)
1,0.020031,(46667)
2,0.018807,(24838)
3,0.011479,(33754)
4,0.006467,(17461)
...,...,...
469,0.008140,"(21903, 45007)"
470,0.007728,"(24852, 45007)"
471,0.006125,"(21137, 45007)"
472,0.005583,"(13176, 39928)"


In [10]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(13176),(21903),0.143778,0.093295,0.020056,0.139494,1.495189,1.0,0.006642,1.053688,0.386802,0.092417,0.050952,0.177234
1,(21903),(13176),0.093295,0.143778,0.020056,0.214975,1.495189,1.0,0.006642,1.090694,0.365266,0.092417,0.083153,0.177234
2,(24852),(21903),0.180509,0.093295,0.020163,0.111698,1.197254,1.0,0.003322,1.020717,0.201046,0.079492,0.020296,0.163907
3,(21903),(24852),0.093295,0.180509,0.020163,0.216116,1.197254,1.0,0.003322,1.045423,0.181708,0.079492,0.043449,0.163907
4,(21137),(21903),0.100894,0.093295,0.014709,0.145786,1.562634,1.0,0.005296,1.061449,0.400459,0.081953,0.057892,0.151723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,(45007),(21137),0.040961,0.100894,0.006125,0.149523,1.481984,1.0,0.001992,1.057179,0.339120,0.045124,0.054086,0.105113
270,(13176),(39928),0.143778,0.019302,0.005583,0.038832,2.011850,1.0,0.002808,1.020320,0.587400,0.035450,0.019915,0.164046
271,(39928),(13176),0.019302,0.143778,0.005583,0.289259,2.011850,1.0,0.002808,1.204690,0.512844,0.035450,0.169911,0.164046
272,(13176),(8174),0.143778,0.016578,0.005451,0.037915,2.287007,1.0,0.003068,1.022177,0.657244,0.035191,0.021696,0.183368


In [47]:
# We only want 1 item predicting 1 item for a simple "Bought Together" UI
simple_rules = rules[(rules['antecedents'].apply(len) == 1) & 
                     (rules['consequents'].apply(len) == 1)].copy()

# Extract the ID from the frozenset
simple_rules['product_id'] = simple_rules['antecedents'].apply(lambda x: list(x)[0])
simple_rules['recommendation_id'] = simple_rules['consequents'].apply(lambda x: list(x)[0])


# Lift tells us the strength, and Confidence tells us the probability
final_recommendations = simple_rules[['product_id', 'recommendation_id', 'confidence', 'lift']]

# 4. Sort to keep only the best 3 recommendations per product
final_recommendations = (final_recommendations
                         .sort_values(['product_id', 'lift'], ascending=[True, False])
                         .groupby('product_id')
                         .head(3))

In [48]:
print(len(final_recommendations))
final_recommendations.head()

152


,product_id,recommendation_id,confidence,lift
796,4605,8518,0.182796,2.188780
799,4605,24964,0.161290,2.000872
790,4605,47766,0.279570,1.984370
406,5450,24852,0.373333,2.106885
405,5450,47626,0.186667,1.836000


In [14]:
all_products = pd.read_csv('data/instacart/products.csv')

In [15]:
all_products[all_products["product_id"] == 5077]

,product_id,product_name,aisle_id,department_id
5076,5077,100% Whole Wheat Bread,112,3


In [16]:
all_products[all_products["product_id"] == 24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [17]:
order_with_dept = order_df.merge(
    all_products[['product_id', 'department_id']], 
    on='product_id', 
    how='left'
)

# Now count the values
dept_counts = order_with_dept['department_id'].value_counts()
print(dept_counts)

department_id
4     1403772
16     562720
7      190024
19     128629
1      121925
13      84681
20      82415
15      71248
3       69376
12      57563
9       46950
14      27238
17      16728
18       8999
5        4142
6        3106
10       2685
21       1377
11       1000
Name: count, dtype: int64


In [18]:
# Check how many categories have at least one recommendation
covered_depts = all_products[all_products['product_id'].isin(rules['antecedents'].apply(lambda x: list(x)[0]))]['department_id'].nunique()
print(f"Recommendations cover {covered_depts} out of {len(dept_counts)} categories.")

Recommendations cover 7 out of 19 categories.


### This other half is a way to generate Data for my fullstack ecommerce app

This app uses Amazon data from Kaggle for product data, but the product does not match the Instacart.

I want to add these recommendations to the product page, so I manipulated the category ID of amazon product to use instacart category id

Since most of the products in the Instacart categories are mostly Groceries, I used the product from the category Grocery_and_Gourmet_Food.

In [20]:
from datasets import load_dataset
# load data from amazon
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Grocery_and_Gourmet_Food", split="full", trust_remote_code=True)
print(dataset[4])

{'main_category': 'Grocery', 'title': 'Messmer Peppermint 25 bags (6er pack)', 'average_rating': 3.5, 'rating_number': 5, 'features': [], 'description': [], 'price': '29.99', 'images': {'hi_res': [None, None], 'large': ['https://m.media-amazon.com/images/I/51tW33sFp6L.jpg', 'https://m.media-amazon.com/images/I/411mYdC5XOL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/51tW33sFp6L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/411mYdC5XOL._SX38_SY50_CR,0,0,38,50_.jpg'], 'variant': ['MAIN', 'PT01']}, 'videos': {'title': [], 'url': [], 'user_id': []}, 'store': 'Messmer', 'categories': [], 'details': '{"Brand": "Messmer", "Item Form": "Teabags", "Unit Count": "1.00 Count", "Package Information": "Bag", "Number of Pieces": "6", "Is Discontinued By Manufacturer": "No", "Package Dimensions": "13.19 x 10.39 x 3.27 inches; 1.37 Pounds"}', 'parent_asin': 'B06X9DC27H', 'bought_together': None, 'subtitle': None, 'author': None}


In [21]:
filtered_data = []
target_count = 1000

for item in dataset:
    # Check if all required fields exist and are not empty
    has_images = item.get('images') and len(item['images']) > 0
    has_desc = item.get('description') and len(item['description']) > 0
    has_price = item.get('price') and item['price'] != 'None'
    
    if has_images and has_price and has_desc:
        filtered_data.append({
            "name": item['title'],
            "price": item['price'],
            "description": item['description'],
            # Amazon stores images as a list of dicts; we take the 'large' or first URL
            "image_url": item['images'],
        })
        
    if len(filtered_data) >= target_count:
        break

# Create your clean DataFrame
amazon_1000_df = pd.DataFrame(filtered_data)
print(f"Successfully collected {len(amazon_1000_df)} products.")

Successfully collected 1000 products.


In [22]:
#Get the 1000 most ordered product ids from the orders from instacart
ordered_products = pd.read_csv('data/instacart/order_products__prior.csv')
top_ids = ordered_products['product_id'].value_counts().head(1000).index
top_ids

Index([24852, 13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209, 27845,
       ...
        4945, 13966,  7746, 35199, 45190, 18023, 20378, 43014,  3339, 27020],
      dtype='int64', name='product_id', length=1000)

In [23]:
all_products = pd.read_csv('data/instacart/products.csv')

# select products that are most ordered
instacart_products = all_products[all_products['product_id'].isin(top_ids)]

In [24]:
instacart_products = instacart_products.reset_index(drop=True)
amazon_1000_df = amazon_1000_df.reset_index(drop=True)

# concat will align them perfectly row-by-row
mock_products = pd.concat([instacart_products, amazon_1000_df], axis=1)

# Clean up
if "aisle_id" in mock_products.columns:
    mock_products.drop(columns=["aisle_id"], inplace=True)

mock_products.rename(columns={"department_id": "category_id"}, inplace=True)

print(f"Final row count: {len(mock_products)}") 

Final row count: 1000


In [26]:
## Save the mock data to be populated in the database
compression_opts = dict(method='zip',
                        archive_name='mock.csv')
mock_products.to_csv('../main-data/mock.zip', index=False,
          compression=compression_opts)   

The previous analysis did not consider the products based on categories, there is a kind of bias because the order data has more order from about 7 categories. the model is only able to capture this 7 categories instead.

This new model is going to consider the categories and try to ensure there is recommendation from each categories

In [54]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

all_category_rules = []
dept_ids = mock_products['category_id'].unique()

for dept in dept_ids:
    # Identify products in this department
    dept_prod_ids = mock_products[mock_products['category_id'] == dept]['product_id'].tolist()
    
    # Get ONLY the orders that contain at least one item from this department
    relevant_order_ids = order_df[order_df['product_id'].isin(dept_prod_ids)]['order_id'].unique()
    
    # Filter the main order_df to get the FULL baskets for those orders
    # We want to see what else people bought ALONG WITH the department items
    subset_orders = order_df[order_df['order_id'].isin(relevant_order_ids)]
    
    # Group into a list of lists (Baskets)
    baskets = subset_orders.groupby('order_id')['product_id'].apply(list).tolist()
    
    # Transaction Encoding
    te = TransactionEncoder()
    te_ary = te.fit(baskets).transform(baskets)
    dept_basket_sets = pd.DataFrame(te_ary, columns=te.columns_)
    
    # Run FP-Growth
    try:
        # We use a very low min_support because we are focusing on specific category pairs
        frequent_itemsets = fpgrowth(dept_basket_sets, min_support=0.01, use_colnames=True)
        
        if not frequent_itemsets.empty:
            rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
            
            # Filter: We only want rules where the 'antecedent' is a product from the current department
            # This ensures we get specific recommendations for the category page
            top_dept_rules = rules.sort_values('lift', ascending=False).head(10)
            all_category_rules.append(top_dept_rules)
            
            print(f"Dept {dept}: Found {len(top_dept_rules)} rules")
    except Exception as e:
        print(f"Dept {dept} failed: {e}")
        continue

# Combine all results
if all_category_rules:
    final_rules = pd.concat(all_category_rules, ignore_index=True)

Dept 14: Found 10 rules
Dept 4: Found 10 rules
Dept 7: Found 10 rules
Dept 19: Found 10 rules
Dept 1: Found 10 rules
Dept 16: Found 10 rules
Dept 15: Found 10 rules
Dept 9: Found 10 rules
Dept 17: Found 10 rules
Dept 5: Found 8 rules
Dept 6: Found 10 rules
Dept 18: Found 10 rules
Dept 20: Found 10 rules
Dept 3: Found 10 rules
Dept 13: Found 10 rules
Dept 12: Found 10 rules
Dept 10: Found 10 rules
Dept 11: Found 10 rules
Dept 21: Found 10 rules


In [55]:
final_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(26209),(47626),0.054848,0.059422,0.010883,0.198418,3.339143,1.0,0.007624,1.173403,0.741174,0.105263,0.147778,0.190782
1,(47626),(26209),0.059422,0.054848,0.010883,0.183145,3.339143,1.0,0.007624,1.157063,0.744778,0.105263,0.135743,0.190782
2,(47209),(5876),0.095935,0.035803,0.010765,0.112207,3.134015,1.0,0.007330,1.086061,0.753176,0.088983,0.079241,0.206434
3,(5876),(47209),0.035803,0.095935,0.010765,0.300661,3.134015,1.0,0.007330,1.292742,0.706205,0.088983,0.226451,0.206434
4,(47766),(26209),0.076338,0.054848,0.010331,0.135331,2.467375,1.0,0.006144,1.093079,0.643862,0.085481,0.085153,0.161842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,(28842),"(26209, 41149)",0.026144,0.107480,0.012346,0.472222,4.393581,1.0,0.009536,1.691090,0.793131,0.101796,0.408666,0.293544
184,(27966),"(17948, 41149)",0.079158,0.036311,0.010893,0.137615,3.789908,1.0,0.008019,1.117469,0.799422,0.104167,0.105121,0.218807
185,(27966),(17948),0.079158,0.036311,0.010893,0.137615,3.789908,1.0,0.008019,1.117469,0.799422,0.104167,0.105121,0.218807
186,"(17948, 41149)",(27966),0.036311,0.079158,0.010893,0.300000,3.789908,1.0,0.008019,1.315489,0.763878,0.104167,0.239826,0.218807


In [56]:
# Sort and Group using the original frozensets
top_recommendations = (final_rules
                        .sort_values(['antecedents', 'lift'], ascending=[True, False])
                        .groupby('antecedents')
                        .head(3))

#convert to lists for your API/JSON
top_recommendations['product_id'] = top_recommendations['antecedents'].apply(list)
top_recommendations['recommendation_id'] = top_recommendations['consequents'].apply(list)


api_recommendations = top_recommendations[['product_id', 'recommendation_id', 'confidence', 'lift']]

In [57]:
api_recommendations

,product_id,recommendation_id,confidence,lift
98,[26209],"[2450, 31717]",0.149660,5.038007
179,[26209],[28842],0.114865,4.393581
182,[26209],"[28842, 41149]",0.114865,4.393581
99,"[2450, 31717]",[26209],0.478261,5.038007
100,"[2450, 47766]","[26209, 24852]",0.124000,4.861114
...,...,...,...,...
91,[45190],[26209],0.109510,1.534846
84,[30391],[47209],0.281205,2.526948
37,[30391],[47209],0.254455,2.501365
88,[30391],[21903],0.249641,2.397019


In [58]:
## Save the recommendation data as a file
api_recommendations.to_csv('../main-data/bought-together.csv', index=False)   

In [34]:
mock_products[mock_products["product_id"] == 4962]

,product_id,product_name,category_id,name,price,description,image_url
85,4962,Yotoddler Organic Pear Spinach Mango Yogurt,16,Hawaiian Sun Guava Jam 10-ounce Jar,19.98,"[""Pretty in Pink,"" the fruit of Hawaii grown g...","{'hi_res': [None], 'large': ['https://m.media-..."


In [35]:
instacart_products[instacart_products["product_id"] == 38544]

,product_id,product_name,aisle_id,department_id
762,38544,Organic Whole Milk Strawberry Beet Berry Yogur...,120,16
